In [17]:
# This code is adapted from https://github.com/AXinx/ELBD

# Load the data.
import pandas as pd
import numpy as np
labeled_data = pd.read_pickle('labeled_data.pkl')
labeled_data

,timestamp,label,carts_response_time,carts_cpu_usage,carts_memory_usage,carts_network_receive,carts_network_transmit,catalogue_response_time,catalogue_cpu_usage,catalogue_memory_usage,...,shipping_response_time,shipping_cpu_usage,shipping_memory_usage,shipping_network_receive,shipping_network_transmit,user_response_time,user_cpu_usage,user_memory_usage,user_network_receive,user_network_transmit
0,2023-09-12 07:10:00,0,12.476340940120815,0.18547890088321844,294682624,420056.65241080325,941.1799168082489,12.437493082457138,0.024590561765954505,6643712,...,2.1944444444444446,0.0052111929537415535,270745600,7956.755630481314,12.539541227056619,8.95863983344905,0.04435522920582362,9068544,182688.9577416301,367.2433083742146
1,2023-09-12 07:10:05,0,12.041813036690137,0.18547890088321844,294682624,280896.1415444082,644.1618677138906,10.741431730237572,0.013942173067587412,6643712,...,2.1944444444444446,0.0036999904237644504,270745600,5002.824262536873,7.921533923303835,8.739691614691575,0.04435522920582362,9072640,182688.9577416301,367.2433083742146
2,2023-09-12 07:10:10,0,12.041813036690137,0.18547890088321842,294682624,400070.4653230587,917.4570243034973,10.741431730237572,0.01595301507632577,6643712,...,2.1944444444444446,0.005300124306459976,270741504,5665.722175923413,8.971174692799103,8.739691614691575,0.04435522920582362,9072640,182688.9577416301,367.2433083742146
3,2023-09-12 07:10:15,0,12.041813036690137,0.18547890088321844,294682624,400070.4653230587,917.4570243034973,10.10742904841405,0.017963857085064126,6643712,...,2.1944444444444446,0.005300124306459976,270741504,6328.620089309952,10.02081546229437,8.61599513720038,0.04435522920582362,9072640,182688.9577416301,367.2433083742146
4,2023-09-12 07:10:20,0,12.041813036690137,0.18547890088321844,294682624,400070.4653230587,917.4570243034973,10.10742904841405,0.019067005011057993,6643712,...,2.1944444444444446,0.005300124306459976,270741504,5847.223624608648,9.258566335486908,8.61599513720038,0.04435522920582362,9072640,182688.9577416301,367.2433083742146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7556,2023-09-12 17:39:40,0,20.82031641043803,0.19104964232241167,317607936,443524.9746633851,1012.0529897205735,21.12278270509978,0.02649505388578243,9170944,...,2.0647058823515185,0.003925517166390261,302043136,4924.353845297719,8.001969949916528,13.088017955801233,0.04286331333640354,19046400,186938.31061952727,381.6906972150898
7557,2023-09-12 17:39:45,0,20.82031641043803,0.19104964232241167,317607936,443524.9746633851,1012.0529897205735,21.44362934363181,0.025807625430748833,9179136,...,2.0647058823515185,0.003925517166390261,302043136,5549.335295863478,9.017551474680024,13.054920580108824,0.04286331333640354,19046400,186938.31061952727,381.6906972150898
7558,2023-09-12 17:39:50,0,20.82031641043803,0.19104964232241167,317607936,443524.97466338496,1012.0529897205734,21.44362934363181,0.025807625430748833,9179136,...,2.0647058823515185,0.003925517166390261,302043136,5817.827327026526,9.45384529771842,13.054920580108824,0.04286331333640354,19046400,186938.3106195273,381.6906972150899
7559,2023-09-12 17:39:55,0,20.82031641043803,0.19104964232241167,317607936,443524.97466338496,1012.0529897205734,21.44362934363181,0.025807625430748833,9179136,...,2.1396551724125383,0.003925517166390261,302043136,5192.8458764607685,8.438263772954926,13.054920580108824,0.04254098554611217,19054592,186938.3106195273,381.6906972150899


## Normalize

In [18]:
# Normalize the data.
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = pd.DataFrame(scaler.fit_transform(labeled_data.drop(['timestamp'], axis=1)))
normalized_data.columns = labeled_data.columns[1:]
normalized_labeled_data = pd.merge(labeled_data.timestamp, normalized_data, left_index=True, right_index=True, how='left')
normalized_labeled_data.fillna(method='ffill', inplace=True)  # Forward fill.
normalized_labeled_data.fillna(method='bfill', inplace=True)  # Backward fill any remaining NaNs.
normalized_labeled_data

,timestamp,label,carts_response_time,carts_cpu_usage,carts_memory_usage,carts_network_receive,carts_network_transmit,catalogue_response_time,catalogue_cpu_usage,catalogue_memory_usage,...,shipping_response_time,shipping_cpu_usage,shipping_memory_usage,shipping_network_receive,shipping_network_transmit,user_response_time,user_cpu_usage,user_memory_usage,user_network_receive,user_network_transmit
0,2023-09-12 07:10:00,0.0,0.000082,0.179846,0.000871,0.406793,0.463301,0.003304,0.032006,0.000032,...,0.004848,0.004752,0.002651,0.241949,0.332955,0.002224,0.048603,0.000000,0.886923,0.858229
1,2023-09-12 07:10:05,0.0,0.000049,0.179846,0.000871,0.266304,0.310241,0.002549,0.017841,0.000032,...,0.004848,0.003207,0.002651,0.149831,0.193231,0.002086,0.048603,0.000004,0.886923,0.858229
2,2023-09-12 07:10:10,0.0,0.000049,0.179846,0.000871,0.386616,0.451076,0.002549,0.020516,0.000032,...,0.004848,0.004843,0.002648,0.170504,0.224990,0.002086,0.048603,0.000004,0.886923,0.858229
3,2023-09-12 07:10:15,0.0,0.000049,0.179846,0.000871,0.386616,0.451076,0.002267,0.023191,0.000032,...,0.004848,0.004843,0.002648,0.191176,0.256748,0.002009,0.048603,0.000004,0.886923,0.858229
4,2023-09-12 07:10:20,0.0,0.000049,0.179846,0.000871,0.386616,0.451076,0.002267,0.024658,0.000032,...,0.004848,0.004843,0.002648,0.176164,0.233685,0.002009,0.048603,0.000004,0.886923,0.858229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7556,2023-09-12 17:39:40,0.0,0.000700,0.185650,0.067643,0.430486,0.499823,0.007171,0.034540,0.002483,...,0.004139,0.003437,0.031105,0.147384,0.195665,0.004810,0.046879,0.009616,0.908189,0.893271
7557,2023-09-12 17:39:45,0.0,0.000700,0.185650,0.067643,0.430486,0.499823,0.007314,0.033625,0.002491,...,0.004139,0.003437,0.031105,0.166874,0.226393,0.004790,0.046879,0.009616,0.908189,0.893271
7558,2023-09-12 17:39:50,0.0,0.000700,0.185650,0.067643,0.430486,0.499823,0.007314,0.033625,0.002491,...,0.004139,0.003437,0.031105,0.175247,0.239594,0.004790,0.046879,0.009616,0.908189,0.893271
7559,2023-09-12 17:39:55,0.0,0.000700,0.185650,0.067643,0.430486,0.499823,0.007314,0.033625,0.002491,...,0.004549,0.003437,0.031105,0.155757,0.208866,0.004790,0.046506,0.009624,0.908189,0.893271


## Ensemble

In [19]:
from __future__ import division
from __future__ import print_function

from time import time
# Supress warnings for clean output.
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn import metrics

from pyod.models.combination import aom, moa, average, maximization, majority_vote
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

In [20]:
def pred_labels(y_test, test_scores):
    fpr, tpr, thresholds = roc_curve(y_test, test_scores)
    cutoff = thresholds[np.argmax(tpr - fpr)]
    pred_label = []
    for each in test_scores:
        if each > cutoff:
            pred_label.append(1)
        else:
            pred_label.append(0)    
    return pred_label

In [21]:
def cal_eval(label, pred_label, scores):
    pr = round(metrics.precision_score(label, pred_label, average='macro'), ndigits=4)
    re = round(metrics.recall_score(label, pred_label, average='macro'), ndigits=4)
    f1 = round(metrics.f1_score(label, pred_label, average='macro'), ndigits=4)
    roc = round(roc_auc_score(label, scores), ndigits=4)
    return pr, re, f1, roc

In [22]:
X_train = normalized_labeled_data.iloc[:, 1:]
y_train = normalized_labeled_data['label']
X_test = normalized_labeled_data.iloc[:, 1:]
y_test = normalized_labeled_data['label']

In [23]:
outliers_fraction = np.count_nonzero(y_train) / len(y_train)
random_state = np.random.RandomState(42)
classifiers = {
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction)
}

n_clf = len(classifiers)
train_scores = np.zeros([X_train.shape[0], n_clf])
test_scores = np.zeros([X_test.shape[0], n_clf])
p_labels = np.zeros([X_test.shape[0], n_clf])

i = 0
train_duration = 0
test_duration = 0
for clf_name, clf in classifiers.items():
    t0 = time()
    clf.fit(X_train)
    t1 = time()
    test_sccore = clf.decision_function(X_test)
    t_t = time()
    test_mean_score = np.nanmean(test_scores)
    test_scores[np.isnan(test_scores)] = test_mean_score
    test_scores[:, i] = test_sccore
    p_labels[:, i] = pred_labels(y_test, test_sccore)
    i += 1
    print('Base detector %i is fitted for prediction' % i)
    train_duration += round(t1 - t0, ndigits=4)
    test_duration += round(t_t - t1, ndigits=4)
    
# standardize test score
test_scores_norm = standardizer(test_scores)
print(train_duration)

Base detector 1 is fitted for prediction
Base detector 2 is fitted for prediction
Base detector 3 is fitted for prediction
Base detector 4 is fitted for prediction
17.345399999999998


## MLP

In [24]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers
from keras.datasets import imdb
from time import time

In [25]:
train_x, test_x, train_y, test_y = train_test_split(test_scores_norm, y_test, test_size=0.9, random_state=random_state)

In [26]:
def ensemble_mlp(df_columns):
    enm_mlp_df = pd.DataFrame(columns=df_columns)

    t2 = time()
    model = models.Sequential()
    # Input Layer
    model.add(layers.Dense(4, activation = "relu", input_shape=(len(train_y), 4)))
    model.add(layers.Dense(20, activation = "relu"))
    model.add(layers.Dense(20, activation = "relu"))
    # Output Layer
    model.add(layers.Dense(1, activation = "sigmoid"))
    model.summary()
    # Compile the model.
    model.compile(
        optimizer = "adam",
        loss = "binary_crossentropy",
        metrics = ["accuracy"]
    )
    model.fit(
     train_x, train_y,
     epochs = 100,
     batch_size = 20,
     validation_data = (train_x, train_y)
    )
    t3 = time()
    pred_test = model.predict(test_scores_norm)
    t4 = time()
    train_time = t3 - t2
    test_time = t4 - t3
    train_time_mlp = round(train_duration + train_time, ndigits=4)
    test_time_mlp = round(test_duration + test_time, ndigits=4)
    
    pred_nn = []
    for each in pred_test:
        if each[0] > 0.5:
            pred_nn.append(1)
        else:
            pred_nn.append(0)
    pr_mlp, re_mlp, f1_mlp, roc_mlp = cal_eval(y_test, pred_nn, pred_test)
    enm_mlp = pd.DataFrame([pr_mlp, re_mlp, f1_mlp, roc_mlp, train_time_mlp, test_time_mlp]).transpose()
    enm_mlp.columns = df_columns
    enm_mlp_df = pd.concat([enm_mlp_df, enm_mlp], axis=0)
    return pred_nn, enm_mlp_df, model

In [27]:
df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
pred_nn, enm_mlp_df, model = ensemble_mlp(df_columns)
# Display the evaluation results.
print(enm_mlp_df)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 756, 4)            20        
_________________________________________________________________
dense_1 (Dense)              (None, 756, 20)           100       
_________________________________________________________________
dense_2 (Dense)              (None, 756, 20)           420       
_________________________________________________________________
dense_3 (Dense)              (None, 756, 1)            21        
Total params: 561
Trainable params: 561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
38/38 [==============================] - 41s 41ms/step - loss: 0.6339 - accuracy: 0.8458 - val_loss: 0.5445 - val_accuracy: 0.7857
Epoch 2/100
38/38 [==============================] - 0s 6ms/step - loss: 0.5238 - accuracy: 0.7717 - val_loss: 0.

In [28]:
pred_nn

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,


In [91]:
df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
t_enm_mlp_df = pd.DataFrame(columns=df_columns)
for i in range(10):
    print(i)
    enm_mlp, model = ensemble_mlp()
    t_enm_mlp_df = pd.concat([t_enm_mlp_df, enm_mlp], axis=0, ignore_index=True)

0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 756, 4)            20        
_________________________________________________________________
dense_1 (Dense)              (None, 756, 20)           100       
_________________________________________________________________
dense_2 (Dense)              (None, 756, 20)           420       
_________________________________________________________________
dense_3 (Dense)              (None, 756, 1)            21        
Total params: 561
Trainable params: 561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
38/38 [==============================] - 14s 13ms/step - loss: 0.6655 - accuracy: 0.6124 - val_loss: 0.6188 - val_accuracy: 0.7923
Epoch 2/100
38/38 [==============================] - 0s 2ms/step - loss: 0.6151 - accuracy: 0.7754 - val_loss: 

In [92]:
print('Ensemble mlp')
metrics_name = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
mean_mlp = []
std_mlp = []
for each in metrics_name:
    mean_mlp.append(round(t_enm_mlp_df[each].mean(), ndigits=4))
    std_mlp.append(round(t_enm_mlp_df[each].std(), ndigits=4))
print(mean_mlp)
print(std_mlp)
t_enm_mlp_df

Ensemble mlp
[0.9347, 0.9225, 0.9278, 0.9784, 18.1623, 5.9506]
[0.0153, 0.0236, 0.0197, 0.008, 3.7507, 0.0604]


,Precision,Recall,F1 score,AUC,Train time,Test time
0,0.9127,0.9017,0.9068,0.9689,28.3825,6.1072
1,0.9247,0.9234,0.9240,0.9781,14.8871,5.9066
2,0.9243,0.9018,0.9116,0.9777,15.5696,5.9029
3,0.9491,0.9464,0.9477,0.9843,16.5245,5.9096
4,0.9434,0.9492,0.9462,0.9840,17.2114,5.9629
5,0.9206,0.8942,0.9054,0.9662,18.3668,5.9325
6,0.9496,0.9385,0.9437,0.9848,17.6673,5.9777
7,0.9215,0.8899,0.9029,0.9684,17.7631,5.9203
8,0.9470,0.9325,0.9391,0.9821,17.8507,5.9333
9,0.9541,0.9478,0.9508,0.9893,17.4003,5.9530
